In [1]:

import time
import os

import numpy as np
from sklearn.metrics import roc_auc_score
import torch
from torch import nn, Tensor, optim
import torch.nn.functional as F
import util

from pertubate import FadeMovingAverage
from NbeatsODE.model import BeatsODE
import argparse
import torch
import torch.nn.functional as F
from torch import nn, Tensor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
parser = argparse.ArgumentParser()

"""
"""

parser.add_argument('--device', type=str, default='cuda', help='device to run the model on')
parser.add_argument('--data', type=str, default='store/PEMS-BAY', help='data path')
parser.add_argument('--adj_data', type=str, default='store/adj_mx_bay.pkl', help='adj data path')

parser.add_argument('--epochs', type=int, default=15, help='')
parser.add_argument('--batch_size', type=int, default=16, help='batch size')

parser.add_argument('--learning_rate', type=float, default=0.001, help='learning rate')
parser.add_argument('--weight_decay', type=float, default=0.0001, help='weight decay rate')

parser.add_argument('--dropout', type=float, default=0.3, help='dropout rate')
parser.add_argument('--print_every', type=int, default=50, help='')
parser.add_argument('--save', type=str, default='saved_models/BeatsODE', help='save path')
parser.add_argument('--log', type=str, default='log/dynamask', help='save path')

parser.add_argument('--blackbox_file', type=str, default='save_blackbox/G_T_model_15.pth', help='blackbox .pth file')

parser.add_argument('--num_nodes', type=int, default=207, help='number of nodes')
parser.add_argument('--timestep', type=str, default=12, help='time step')
parser.add_argument('--input_dim', type=str, default=2, help='channels')
parser.add_argument('--output_dim', type=str, default=1, help='channels')
parser.add_argument('--samples', type=str, default=10, help='samples data')
parser.add_argument('--sample_iters', type=str, default=1000, help='___')
parser.add_argument('--initial_mask_coeff', type=str, default=0.5, help='___')
parser.add_argument('--reg_coeff', type=str, default=0.07, help='___')

args = parser.parse_args('')


In [3]:
def check(sample, arr):
    for i, index in enumerate(arr): 
        sample[:, i, index, :]= 0.0

sample = np.load('./saved_models/dynamask/sample_METR_LA_0.npz')['arr_0']



device = torch.device(args.device)
_, _, adj_mx = util.load_adj('store/adj_mx.pkl')

adj_mx = torch.tensor(adj_mx).to(device)


model = BeatsODE(in_dim=2, out_dim=1, seq_len=12).to(device)

model.load_state_dict(torch.load('save_blackbox/G_T_model_15.pth'))


print("start training...", flush=True)
his_loss = []
val_time = []
train_time = []
best_epoch = 0


mask = np.load('./saved_models/dynamask/saliency_METR_LA_0.npz')['arr_0']
# for i, sample in enumerate(samples):
sample= torch.Tensor(sample).unsqueeze(0)
trainx = torch.Tensor(sample).to(device)
print(trainx.shape)

output = model(trainx.transpose(1, 3), adj_mx)
output = output[:, :, :, 0:1]
output= output.transpose(-3, -1)
mask= mask[0, 0, :, :]
mask= mask.T
mask = torch.Tensor(mask)
top_k = torch.topk(mask, 15, dim=1, sorted= True)
index_ = top_k.indices
arr= []
for i in range(index_.shape[0]):
        row= np.full((207, ), True)
        row[index_[i]]= False
        arr.append(row)
arr = np.array(arr)

check(sample, arr)
trainx = torch.Tensor(sample).to(device)
output2 = model(trainx.transpose(1, 3), adj_mx)
output2 = output2[:, :, :, 0:1]
output2= output2.transpose(-3, -1)
MAE= util.masked_mae(output2, output, 0.0)
MAPE= util.masked_mape(output2, output, 0.0)
RMSE= util.masked_rmse(output2, output, 0.0)


print(f'MAE: {MAE:.4f}, MAPE: {MAPE:.4f}, RMSE: {RMSE:.4f} \n')

        


start training...
torch.Size([1, 12, 207, 2])
MAE: 0.5915, MAPE: 0.6341, RMSE: 0.7255 



In [3]:
def check(sample, arr):
    for i, index in enumerate(arr): 
        sample[:, i, index, :]= 0.0


def normalize_tensor(data):
    min_val = data.min()
    max_val = data.max()
    normalized_data = (data - min_val) / (max_val - min_val)
    # X_scaled = normalized_data * (max_val - min_val) + min_val

    return normalized_data


sample = np.load('./saved_models/METR_LA/sample_METR_LA_1.npz')['arr_0']



device = torch.device(args.device)
_, _, adj_mx = util.load_adj('store/adj_mx.pkl')

adj_mx = torch.tensor(adj_mx).to(device)


model = BeatsODE(in_dim=2, out_dim=1, seq_len=12).to(device)

model.load_state_dict(torch.load('save_blackbox/G_T_model_15.pth'))


print("start training...", flush=True)


results = []


mask = np.load('./saved_models/METR_LA/saliency_METR_LA_1.npz')['arr_0']
# for i, sample in enumerate(samples):
sample= torch.Tensor(sample).unsqueeze(0)
trainx = torch.Tensor(sample).to(device)
print(trainx.shape)

mask= mask[0, 0, :, :]
# print(mask.shape)
mask= normalize_tensor(mask)
mask= mask.T
mask = torch.Tensor(mask)
# print("mask: ", mask)
top_k = torch.topk(mask, 30, dim=1, sorted= True)
index_ = top_k.indices
arr= []
for i in range(index_.shape[0]):
        row= np.full((207, ), True)
        row[index_[i]]= False
        arr.append(row)
arr = np.array(arr)

amae= [0]*12
amape= [0]*12
armse= [0]*12
temp = 10
for iter in range(temp):
        for hori in range(12):
                with torch.no_grad():
                        output = model(trainx.transpose(1, 3), adj_mx)
                        output = output[:, hori, :, 0:1]
                        # print(output.shape)

                        

                        check(sample, arr)
                        trainx = torch.Tensor(sample).to(device)
                        output2 = model(trainx.transpose(1, 3), adj_mx)
                        output2 = output2[:, hori, :, 0:1]

                        MAE= util.masked_mae(output2, output, 0.0)
                        MAPE= util.masked_mape(output2, output, 0.0)
                        RMSE= util.masked_rmse(output2, output, 0.0)
                        
                        amae[hori]+= MAE.item()
                        amape[hori]+= MAPE.item()
                        armse[hori]+= RMSE.item()
                        
        print("Done: ", iter)

for hori in range(12):
        print(f'Horizon: {hori + 1} ,MAE: {amae[hori]/temp:.4f}, MAPE: {amape[hori]/temp:.4f}, RMSE: {armse[hori]/temp:.4f} \n')
                # print (output)
                # print("---------------------------------------------")
                # print(output2)
                



        # print(f'Mean:  ,MAE: {np.mean(amae)}, MAPE: {np.mean(amape)}, RMSE: {np.mean(armse)} ')
                
        # sorted_results = sorted(results, key=lambda x: x[0])

        # for result in sorted_results:
        #     hori, MAE, MAPE, RMSE = result
        #     print(f'Horizon: {hori} ,MAE: {MAE:.4f}, MAPE: {MAPE:.4f}, RMSE: {RMSE:.4f} \n')

Unable to load data  store/adj_mx.pkl : [Errno 2] No such file or directory: 'store/adj_mx.pkl'


FileNotFoundError: [Errno 2] No such file or directory: 'store/adj_mx.pkl'

In [50]:
def check(sample, arr):
    for i, index in enumerate(arr): 
        sample[:, i, index, :]= 0.0


def normalize_tensor(data):
    min_val = data.min()
    max_val = data.max()
    normalized_data = (data - min_val) / (max_val - min_val)
    # X_scaled = normalized_data * (max_val - min_val) + min_val

    return normalized_data


sample = np.load('./saved_models/PEMS_BAY/sample_PEMS_BAY_1.npz')['arr_0']



device = torch.device(args.device)
_, _, adj_mx = util.load_adj('store/adj_mx_bay.pkl')

adj_mx = torch.tensor(adj_mx).to(device)


model = BeatsODE(in_dim=2, out_dim=1, seq_len=12).to(device)

model.load_state_dict(torch.load('save_blackbox/G_T_model_15.pth'))


print("start training...", flush=True)


results = []


mask = np.load('./saved_models/PEMS_BAY/saliency_PEMS_BAY_1.npz')['arr_0']
# for i, sample in enumerate(samples):
sample= torch.Tensor(sample).unsqueeze(0)
trainx = torch.Tensor(sample).to(device)
print(trainx.shape)

mask= mask[0, 0, :, :]
# print(mask.shape)
mask= normalize_tensor(mask)
mask= mask.T
mask = torch.Tensor(mask)
# print("mask: ", mask)
top_k = torch.topk(mask, 30, dim=1, sorted= True)
index_ = top_k.indices
arr= []
for i in range(index_.shape[0]):
        row= np.full((325, ), True)
        row[index_[i]]= False
        arr.append(row)
arr = np.array(arr)

amae= [0]*12
amape= [0]*12
armse= [0]*12
temp = 10

for hori in range(12):
        with torch.no_grad():
                output = model(trainx.transpose(1, 3), adj_mx)
                output = output[:, hori, :, 0:1]
                # print(output.shape)

                

                check(sample, arr)
                trainx = torch.Tensor(sample).to(device)
                output2 = model(trainx.transpose(1, 3), adj_mx)
                output2 = output2[:, hori, :, 0:1]

                MAE= util.masked_mae(output2, output, 0.0)
                MAPE= util.masked_mape(output2, output, 0.0)
                RMSE= util.masked_rmse(output2, output, 0.0)
                
 


                print(f'Horizon: {hori + 1} ,MAE: {MAE:.4f}, MAPE: {MAPE:.4f}, RMSE: {RMSE:.4f} \n')
                # print (output)
                # print("---------------------------------------------")
                # print(output2)
                



        # print(f'Mean:  ,MAE: {np.mean(amae)}, MAPE: {np.mean(amape)}, RMSE: {np.mean(armse)} ')
                
        # sorted_results = sorted(results, key=lambda x: x[0])

        # for result in sorted_results:
        #     hori, MAE, MAPE, RMSE = result
        #     print(f'Horizon: {hori} ,MAE: {MAE:.4f}, MAPE: {MAPE:.4f}, RMSE: {RMSE:.4f} \n')

start training...
torch.Size([1, 12, 325, 2])
Horizon: 1 ,MAE: 0.3894, MAPE: 0.7176, RMSE: 0.4482 

Horizon: 2 ,MAE: 0.0202, MAPE: 1.4719, RMSE: 0.0254 

Horizon: 3 ,MAE: 0.0218, MAPE: 1.5451, RMSE: 0.0273 

Horizon: 4 ,MAE: 0.0201, MAPE: 1.4420, RMSE: 0.0251 

Horizon: 5 ,MAE: 0.0197, MAPE: 0.3604, RMSE: 0.0249 

Horizon: 6 ,MAE: 0.0198, MAPE: 2.0017, RMSE: 0.0247 

Horizon: 7 ,MAE: 0.0208, MAPE: 0.8142, RMSE: 0.0263 

Horizon: 8 ,MAE: 0.0196, MAPE: 0.6974, RMSE: 0.0247 

Horizon: 9 ,MAE: 0.0219, MAPE: 0.3718, RMSE: 0.0271 

Horizon: 10 ,MAE: 0.0202, MAPE: 27.3869, RMSE: 0.0256 

Horizon: 11 ,MAE: 0.0192, MAPE: 0.4097, RMSE: 0.0243 

Horizon: 12 ,MAE: 0.0199, MAPE: 0.0877, RMSE: 0.0248 

